In [ ]:
import pandas as pd
import re
import nltk
import numpy as np

In [ ]:
# Using all pickles of the users
non_elite_reviews_pkl = pd.read_pickle('non_elite_reviews.pkl')
real_elite_reviews_pkl= pd.read_pickle('real_elite_reviews.pkl')
slack_elite_reviews_pkl=pd.read_pickle('slack_elite_reviews.pkl')
potential_elite_reviews_pkl=pd.read_pickle('potential_elite_reviews.pkl')

In [ ]:
# Check first 10 rows of the potential users
potential_elite_reviews_pkl.head(100)

In [ ]:
sample_data = potential_elite_reviews_pkl.head(100).append(real_elite_reviews_pkl.head(100)).append(slack_elite_reviews_pkl.head(100)).append(non_elite_reviews_pkl.head(100))

In [ ]:

potential_subset=sample_data['text']
potential_subset

In [ ]:
# Use regular expressions to do a find-and-replace and change the text to lowercase
def filter_alphabets(input_string):
    return re.sub("[^a-zA-Z]", " ", input_string).lower()

result_potential=potential_subset.apply(filter_alphabets)
result_potential.head(2)

In [ ]:
# split the text into words
def split_text(input_text):
    return input_text.split()

In [ ]:
result_potential_1 = result_potential.apply(split_text)

In [ ]:
result_potential_1.head()

In [ ]:
nltk.download()

from nltk.corpus import stopwords
def remove_stopwords(words):
    return [w for w in words if not w in stopwords.words("english")]


In [ ]:
potential_removed_stopwords=result_potential_1.apply(remove_stopwords)
potential_removed_stopwords

In [ ]:
#Join the words back into one string separated by space, 
    # and return the result.
def join_words(words):
    return( " ".join(words))


In [ ]:
potential_removed_stopwords_joined=potential_removed_stopwords.apply(join_words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)
train_data_features = vectorizer.fit_transform(potential_removed_stopwords_joined)
train_data_features = train_data_features.toarray()
train_data_features

In [ ]:
train_data_features.head(10)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, sample_data["category"] )

In [ ]:
import numpy as np
test_reviews = non_elite_reviews_pkl.tail(100)['text']
test_data_features = vectorizer.transform(test_reviews)
np.asarray(test_data_features)
print "Predicting test labels...\n"
result = forest.predict(test_data_features)
result

